# SuPreMo get_seq walkthrough

In this walkthrough, we will run get_seq on CTCF binding site deletions, which we generated in [custom_perturbations.ipynb](https://github.com/ketringjoni/Akita_variant_scoring/blob/main/walkthroughs/custom_perturbations.ipynb).

To do so, we've run the following command (the output of the following cell) in the terminal. By changing the variables below, you can get a command that fits your custom set of perturbations. Note: there are more arguments available that are not included here.  

In [31]:
import pandas as pd
import numpy as np
import os
import pysam
from collections import Counter

In [19]:
in_file = '../test_data/custom_perturbations/input/CTCF_del_symb_alleles.txt' # A set of 50 CTCF deletions
shift_by = '-1 0 1' # Shift sequences by 1 bp left and right
revcomp = 'add_revcomp' # Also score the reverse complement of all sequences
file = 'CTCF_del' # Output files prefix
directory = '../test_data/custom_perturbations/output' # Output directory
seq_length = 1048576 # deafault sequence length for Akita

print('Run this command in the main directory:\n')
print('python scripts/SuPreMo.py', in_file,
      '--shift_by', shift_by,
      '--revcomp', revcomp,
      '--file', file,
      '--dir', directory,
      '--get_seq')


Run this command in the main directory:

python scripts/SuPreMo.py ../test_data/custom_perturbations/input/CTCF_del_symb_alleles.txt --shift_by -1 0 1 --revcomp add_revcomp --file CTCF_del --dir ../test_data/custom_perturbations/output --get_seq


In [20]:
# Get path to output files
out_file = os.path.join(directory, file)

In [21]:
variants = pd.read_csv(in_file, sep = '\t')

# Read input

import sys
sys.path.insert(0, '../scripts')

import reading_utils
reading_utils.var_set_size = 10000000

variants = reading_utils.read_input(in_file, 0)
variants

,CHROM,POS,REF,ALT,END,SVTYPE,SVLEN
0,chr1,100276250,-,-,100276269,DEL,19
1,chr1,101106697,-,-,101106716,DEL,19
2,chr1,101159421,-,-,101159440,DEL,19
3,chr1,101442377,-,-,101442396,DEL,19
4,chr1,101526743,-,-,101526762,DEL,19
5,chr1,101595702,-,-,101595721,DEL,19
6,chr1,101693506,-,-,101693525,DEL,19
7,chr1,101744879,-,-,101744898,DEL,19
8,chr1,102007853,-,-,102007872,DEL,19
9,chr1,10192988,-,-,10193007,DEL,19


# Filtered out file

In [22]:
# Read filtered out file

filtered_out_file = f'{out_file}_filtered_out'

if os.path.getsize(filtered_out_file) == 0:
    
    print('No perturbations were filtered out.')

else:
    
    with open(filtered_out_file) as f:
        filtered_out = f.read().splitlines()
        
    # Get the number of variants that were filtered out for each type of reason

    print(Counter([x.split(': ')[1] for x in filtered_out]))
    
    
    # Get list of perturbations (input row number) that were filtered out

    filtered_out_rows = np.unique([int(x.split(': ')[0]) for x in filtered_out])

    print(filtered_out_rows[:10])

No perturbations were filtered out.


# Log file

Check the log file for errors and warnings

In [24]:
# Read log file

with open(f'{out_file}_log') as f: # 
    log = f.read().splitlines()

log[:10]

['0 (-1 shift)',
 '0 (-1 shift_revcomp)',
 '0 (0 shift)',
 '0 (0 shift_revcomp)',
 '0 (1 shift)',
 '0 (1 shift_revcomp)',
 '1 (-1 shift)',
 '1 (-1 shift_revcomp)',
 '1 (0 shift)',
 '1 (0 shift_revcomp)']

In [25]:
print('There are',len(log),'log messages so the counts below are out of', len(log))

There are 300 log messages so the counts below are out of 300


## Warnings

In [26]:
# Get perturbations that had warnings

warnings = [x for x in log if 'Warning' in x]

warnings[:10]

[]

In [29]:
# Get the number of each type of warning message

Counter([x.split('Warning: ')[1] for x in warnings])

Counter()

In [32]:
# Get list of perturbations (input row number) that have warnings in at least one of the conditions

warning_rows = np.unique([x.split(' (')[0] for x in warnings])

warning_rows[:10]

array([], dtype=float64)

## Errors 

In [33]:
# Get perturbations that had errors

errors = [x for x in log if 'Error' in x]
error_messages = [x.split('Error: ')[1] for x in errors]

errors[:10]

[]

In [35]:
# These are all the expected error messages

with open('../scripts/get_seq_utils.py', 'r') as f:
    get_seq_error_messages = np.unique([l.split('\'')[1] for l in f if 'raise ValueError' in l])

print('get_seq expected error messages:\n\n', get_seq_error_messages, '\n\n')


get_seq expected error messages:

 ['Alternate sequence generated is not the right length.'
 'Cannot generate 1Mb sequence for this chromosomal rearrangement.'
 'Centromeric variant.' 'N composition greater than 5%.'
 'Reference allele does not match hg38.'
 'Reference sequence generated is not the right length.'
 'SV type not supported.'
 'Sequence for the alternate allele does not match hg38 at REF position.'
 'Unexpected format: BND ALT does not include REF.'
 'Variant larger than set limit.'] 




In [36]:
# Get the number of each type of expected error message

Counter(error_messages)

Counter()

In [37]:
# Get number of get_seq and get_Akita_scores error messages

print(len([x for x in error_messages if x in get_seq_error_messages]), 'get_seq error messages')
print(len([x for x in error_messages if x in get_Akita_scores_error_messages]), 'get_Akita_scores error messages')

0 get_seq error messages
0 get_Akita_scores error messages


In [38]:
# Check if any of these errors are unexpected

[x for x in error_messages if x not in get_seq_error_messages and x not in get_Akita_scores_error_messages]

[]

In [39]:
# Get list of perturbations (input row number) that have errors in at least one of the conditions

error_rows = np.unique([x.split(' (')[0] for x in errors])

error_rows[:10]

array([], dtype=float64)

# Read in sequences

In [9]:
seq_file = out_file + '_sequences.fa'
seq_file = '../temp_output/score_var_results_sequences.fa'

seq_names = pysam.Fastafile(seq_file).references
seq_names[:10]

['0_0_0_[524287_474287]', '0_0_1_[524287_474287]']

In [90]:
# Get sequences

sequences = {}

shift_by = str(0)

for var_index in variants.index:
    for shift in shift_by.split(' '):
        for revcomp in ['']:

            if 'SVLEN' in variants.columns:
                
                SVTYPE = variants.iloc[var_index].SVTYPE
                SVLEN = variants.iloc[var_index].SVLEN
                if SVTYPE == 'DUP':
                    SVLEN = 2*SVLEN
            else:
                SVTYPE, SVLEN = '-', '0'


            if SVTYPE == 'BND':
                alleles = {0: 'REF1', 
                           1: 'REF2',
                           2: 'ALT'}
            else:
                alleles = {0: 'REF', 
                           1: 'ALT'}


            for allele in range(len(alleles)):
                variant_id = f'{var_index}_{shift}{revcomp}_{allele}'

                seq_name = [x for x in seq_names if variant_id in x][0]

                position = int(seq_name.split('[')[1].split(']')[0].split('_')[allele])

                sequence = pysam.Fastafile(seq_file).fetch(seq_name, 0, seq_length).upper()

                sequences.update({str(var_index)+'_'+alleles[allele]:sequence})

                # Look at the region where the variant is in that sequence
                print('var_index =',var_index, '\tshift =', shift, revcomp,
                      '\tstart of variant:', sequence[position:position+SVLEN][:10],
                      '\tend of variant:', sequence[position:position+SVLEN][-10:])

var_index = 0 	shift = 0  	start of variant: TCCCTTATAT 	end of variant: GTCAAGAGTT
var_index = 0 	shift = 0  	start of variant: TCCCTTATAT 	end of variant: ATGTATACAT


In [93]:
sequences.keys()

dict_keys(['0_REF', '0_ALT'])

In [92]:
sequences

{'0_REF': '...',
 '0_ALT': '...'}